In [1]:
# Things to import
import pandas as pd 
import requests
import io

from pandas import ExcelWriter

import lxml.html as lh
from pandas.io.json import json_normalize

Reading the districts and neighbourhood database

In [2]:
xl = pd.ExcelFile("madrid_neigh.xlsx")
xl.sheet_names
[u'madrid_neigh', u'Sheet2', u'Sheet3']
df_barrios = xl.parse("madrid_neigh")
df_barrios.head()

,District,Neighbourhood,Latitude,Longitude
0,Centro,Palacio,40.415,-3.713
1,Centro,Embajadores,40.408,-3.699
2,Centro,Cortes,40.414,-3.698
3,Centro,Justicia,40.423,-3.696
4,Centro,Universidad,40.425,-3.708


Coordinates of the city of Madrid

In [3]:
latitude = 40.416775
longitude = -3.703790
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid are 40.416775, -3.70379.


Credentials for Foursquare

In [4]:
CLIENT_ID = 'DNJZI24LRM02A4TJF0OHDZU4OIZ5VZTEJPN440IEW2ISSKAP' # your Foursquare ID
CLIENT_SECRET = 'PI1KOAVOT5EH2DZW5XEHCSQWWWR1ZEQHHWULYCK1AYGCBN4D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DNJZI24LRM02A4TJF0OHDZU4OIZ5VZTEJPN440IEW2ISSKAP
CLIENT_SECRET:PI1KOAVOT5EH2DZW5XEHCSQWWWR1ZEQHHWULYCK1AYGCBN4D


Checking only one neighbourhood

In [6]:
df_barrios.loc[0, 'Neighbourhood']

'Palacio'

Coordinates recall for the neighbourhood

In [7]:
neighbourhood_latitude = df_barrios.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_barrios.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_barrios.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Palacio are 40.415, -3.713.


URL generation

In [8]:
# type your answer here
LIMIT = 250
radius = 500

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DNJZI24LRM02A4TJF0OHDZU4OIZ5VZTEJPN440IEW2ISSKAP&client_secret=PI1KOAVOT5EH2DZW5XEHCSQWWWR1ZEQHHWULYCK1AYGCBN4D&v=20180605&ll=40.415,-3.713&radius=500&limit=250'

Get the results with from generated URL

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce6736e4c1f6753b6f79ea8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'La Latina',
  'headerFullLocation': 'La Latina, Madrid',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 154,
  'suggestedBounds': {'ne': {'lat': 40.4195000045, 'lng': -3.7071006203134664},
   'sw': {'lat': 40.4104999955, 'lng': -3.7188993796865337}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcda38f964a520523c21e3',
       'name': 'Santa Iglesia Catedral de Santa María la Real de la Almudena (Catedral de la Almudena)',
       'location': {'address': 'C. Bailén, 8-10',
        'lat': 40.41576693264202,
        'lng': -3.7145161628723145,
        

Function that extracts the category of the venue

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Display venues from the previous neighbourhood

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Santa Iglesia Catedral de Santa María la Real ...,Church,40.415767,-3.714516
1,Cervecería La Mayor,Beer Bar,40.415218,-3.712194
2,Mercado Jamón Iberico,Market,40.415309,-3.711633
3,Plaza de la Villa,Historic Site,40.415409,-3.710391
4,Plaza de La Almudena,Plaza,40.416320,-3.713777


Obtain venues from all neighbourhoods

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
madrid_venues = getNearbyVenues(names=df_barrios['Neighbourhood'],
                                   latitudes=df_barrios['Latitude'],
                                   longitudes=df_barrios['Longitude']
             
                               )

Palacio
Embajadores
Cortes
Justicia
Universidad
Sol
Imperial
Acacias
Chopera
Legazpi
Delicias
Palos de Moguer
Atocha
Pacífico
Adelfas
Estrella
Ibiza
Jerónimos
Niño Jesús
Recoletos
Goya
Fuente del Berro
La Guindalera
Lista
Castellana
El Viso
Prosperidad
Ciudad Jardín
Hispanoamérica
Nueva España
Castilla
Bellas Vistas
Cuatro Caminos
Castillejos
Almenara
Valdeacederas
Berruguete
Gaztambide
Arapiles
Trafalgar
Almagro
Ríos Rosas
Vallehermoso
El Pardo
Fuentelarreina
Peñagrande
El Pilar
La Paz
Valverde
Mirasierra
El Goloso
Casa de Campo
Argüelles
Ciudad Universitaria
Valdezarza
Valdemarín
El Plantío
Aravaca
Los Cármenes
Puerta del Ángel
Lucero
Aluche
Campamento
Cuatro Vientos
Las Águilas
Comillas
Opañel
San Isidro
Vista Alegre
Puerta Bonita
Buenavista
Abrantes
Orcasitas
Orcasur
San Fermín
Almendrales
Moscardó
Zofío
Pradolongo
Entrevías
San Diego
Palomeras Bajas
Palomeras Sureste
Portazgo
Numancia
Pavones
Horcajo
Marroquina
Media Legua
Fontarrón
Vinateros
Ventas
Pueblo Nuevo
Quintana
Concepció

Check all the venue categories

In [15]:
madrid_venues['Venue Category'].unique()

array(['Church', 'Beer Bar', 'Market', 'Historic Site', 'Plaza',
       'Tapas Restaurant', 'Paella Restaurant', 'Garden',
       'Ice Cream Shop', 'Restaurant', 'Peruvian Restaurant',
       'Performing Arts Venue', 'Bar', 'Wine Shop', 'Gastropub',
       'Spanish Restaurant', 'Bakery', 'Palace',
       'Vegetarian / Vegan Restaurant', 'Latin American Restaurant',
       'Café', 'Wine Bar', 'French Restaurant', 'Cocktail Bar', 'Park',
       'Opera House', 'Lounge', 'Other Nightlife', 'Mexican Restaurant',
       'Resort', 'Hobby Shop', 'Dumpling Restaurant',
       'Gym / Fitness Center', 'Pie Shop', 'Pastry Shop',
       'Mediterranean Restaurant', 'Ramen Restaurant', 'Music Venue',
       'American Restaurant', 'Hotel', 'Museum', 'Road',
       'Comfort Food Restaurant', 'Pizza Place', 'Coffee Shop', 'Theater',
       'Seafood Restaurant', 'Argentinian Restaurant', 'Beer Garden',
       'Art Gallery', 'Circus', 'Breakfast Spot', 'Record Shop',
       'Souvlaki Shop', 'Bookstore', '

In [15]:
madrid_restbars = madrid_venues['Venue Category'].isin(['Beer Bar', 'Tapas Restaurant', 'Paella Restaurant',
                                                        'Ice Cream Shop', 'Restaurant', 'Peruvian Restaurant',
                                                        'Bar', 'Gastropub','Spanish Restaurant',
                                                        'Vegetarian / Vegan Restaurant', 'Latin American Restaurant',
                                                        'Café', 'Wine Bar', 'French Restaurant', 'Cocktail Bar',
                                                        'Mexican Restaurant', 'Dumpling Restaurant',
                                                        'Mediterranean Restaurant', 'Ramen Restaurant',
                                                        'American Restaurant','Pizza Place',
                                                        'Seafood Restaurant', 'Argentinian Restaurant', 'Beer Garden',
                                                        'Breakfast Spot','Sushi Restaurant',
                                                        'Pet Café', 'Thai Restaurant',
                                                        'Grilled Meat Restaurant', 'Pub', 'Molecular Gastronomy Restaurant',
                                                        'African Restaurant', 'Asian Restaurant',
                                                        'Italian Restaurant', 'Indian Restaurant', 'Theme Restaurant',
                                                        'Japanese Restaurant',
                                                        'Cuban Restaurant', 'Noodle House', 'Creperie'
                                                        'Food & Drink Shop', 'Dessert Shop',
                                                        'Juice Bar','Gay Bar', 'Salad Place', 'Hawaiian Restaurant', 'Burger Joint',
                                                        'Moroccan Restaurant', 'Chinese Restaurant', 'Sandwich Place',
                                                        'Middle Eastern Restaurant', 'Donut Shop','Tea Room',
                                                        'Cajun / Creole Restaurant', 'Cafeteria',
                                                        'Frozen Yogurt Shop', 'Burrito Place',
                                                        'Smoothie Shop', 'Chocolate Shop', 'Bistro',
                                                        'Vietnamese Restaurant', 'Candy Store', 'Arepa Restaurant', 'Snack Place', 
                                                        'Brewery','Diner',
                                                        'Irish Pub', 'Korean Restaurant', 'BBQ Joint', 'Falafel Restaurant', 'Portuguese Restaurant',
                                                        'Salvadoran Restaurant','Fast Food Restaurant',
                                                        'Polish Restaurant', 'Greek Restaurant', 'Cupcake Shop', 'Taverna',
                                                        'Turkish Restaurant','Steakhouse', 'Taco Place','Food Court',
                                                        'South American Restaurant', 'Karaoke Bar','Persian Restaurant',
                                                        'Modern European Restaurant', 'Brazilian Restaurant','Sports Bar',
                                                        'Eastern European Restaurant', 'German Restaurant','Fried Chicken Joint',
                                                        'Lebanese Restaurant', 'Kebab Restaurant','Hot Dog Joint'
                                                       ])

In [16]:
madrid_restbars = madrid_venues[madrid_restbars]

In [17]:
writer = ExcelWriter('madrid_restbars.xlsx')
madrid_restbars.to_excel(writer,'Sheet1')
writer.save()